In [1]:
!pip install unsloth

In [2]:
import pandas as pd
import random
import json
from unsloth import FastLanguageModel
import torch


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [12]:


from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
INPUT_CSV = "/content/drive/MyDrive/lab_test_results_public.csv"
OUTPUT_JSONL = "/content/drive/MyDrive/lab_reports_training_dataset.jsonl"
OUTPUT_JSON = "/content/drive/MyDrive/lab_reports_training_dataset.json"
NUM_ADDITIONAL_SYNTHETIC = 50  # Number of extra synthetic examples to generate



# ============================================================================
# HELPER FUNCTIONS
# ============================================================================



def create_prompt_for_generation(test_name, result, unit, ref_range, comment=""):
    """Create a prompt for LLM to generate explanation"""
    return f"""### Instruction:
You are a medical assistant that explains lab test results in simple, easy-to-understand language for patients without medical background. Be empathetic, clear, and avoid medical jargon.

### Input:
Test Name: {test_name}
Result: {result} {unit}
Reference Range: {ref_range}
{f"Clinical Note: {comment}" if comment else ""}

### Response:
"""


def format_for_training(test_name, result, unit, ref_range, explanation):
    """Format data for training"""
    input_text = f"Test: {test_name} | Result: {result} {unit} (Reference: {ref_range})"
    return {
        "instruction": f"Explain this lab test result in simple terms: {input_text}",
        "output": explanation
    }


# ============================================================================
# STEP 1: Load and print Dataset samples
# ============================================================================

print("=" * 80)
print("STEP 1: Loading and Translating Dataset")
print("=" * 80)

df = pd.read_csv(INPUT_CSV)

print(f"\nData Samples:")
print(df[["Test_Name", "Result", "Unit", "Comment"]].head(3))

# ============================================================================
# STEP 2: Load Unsloth Model for Synthetic Generation
# ============================================================================

print("\n" + "=" * 80)
print("STEP 2: Loading Unsloth Model")
print("=" * 80)

try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(model)
    print("✅ Model loaded successfully")
except Exception as e:
    print(f"⚠️  Warning: Could not load model: {e}")
    print("   Continuing with template-based generation only...")
    model = None
    tokenizer = None



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
STEP 1: Loading and Translating Dataset

Data Samples:
                         Test_Name Result  Unit  \
0                         Ferritin   28.9  ug/L   
1  Glycosylated Hemoglobin (HbA1c)    5.0     %   
2                        Total IgE   1.73  KU/L   

                                   Comment  
0  Iron stores are at a low-moderate level  
1    Long-term blood sugar average is good  
2                         Low allergy risk  

STEP 2: Loading Unsloth Model
==((====))==  Unsloth 2025.11.2: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading i

model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Model loaded successfully


In [15]:
print("\n🔄 Processing original dataset...")
training_data = []

for idx, row in df.iterrows():
    explanation = f"{row['Comment']} {row['Recommended_Followup']}".strip()

    if not explanation:
        explanation = "Your test result has been recorded. Please consult with your healthcare provider for interpretation."

    formatted = format_for_training(
        test_name=row['Test_Name'],
        result=row['Result'],
        unit=row['Unit'],
        ref_range=row['Reference_Range'],
        explanation=explanation
    )
    training_data.append(formatted)

print(f"✅ Processed {len(training_data)} examples from original dataset")


🔄 Processing original dataset...
✅ Processed 27 examples from original dataset


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
An error occurred while loading the file: name 'pd' is not defined


In [18]:
print("\n🧪 Creating synthetic examples...")

synthetic_templates = [
    {
        "test": "Glucose (Fasting)", "unit": "mg/dL", "ref": "70-100",
        "scenarios": [
            (85, "Your fasting blood sugar is in the healthy range. This indicates good blood sugar control. Continue maintaining a balanced diet and regular physical activity."),
            (65, "Your blood sugar is slightly low. This might cause symptoms like shakiness or dizziness. Consider eating small, regular meals throughout the day and discuss with your doctor if this happens frequently."),
            (125, "Your fasting blood sugar is elevated above the normal range. This could indicate prediabetes. Lifestyle changes like diet modification, weight management, and regular exercise are recommended. Follow up with your doctor."),
            (180, "Your blood sugar is significantly high, which may indicate diabetes. This requires medical attention. Your doctor may recommend further testing and discuss treatment options."),
        ]
    },
    {
        "test": "HbA1c", "unit": "%", "ref": "<5.7",
        "scenarios": [
            (5.2, "Your HbA1c shows excellent long-term blood sugar control over the past 2-3 months. Keep up your current lifestyle habits."),
            (6.0, "Your HbA1c is in the prediabetes range (5.7-6.4%). This means your average blood sugar has been higher than normal. Now is a good time to make lifestyle changes to prevent type 2 diabetes."),
            (7.5, "Your HbA1c indicates diabetes (6.5% or higher). This reflects your average blood sugar over the past few months. Work closely with your healthcare team to develop a management plan."),
        ]
    },
    {
        "test": "Vitamin D", "unit": "ng/mL", "ref": "30-100",
        "scenarios": [
            (45, "Your Vitamin D level is in the optimal range. Vitamin D is important for bone health, immune function, and mood."),
            (15, "Your Vitamin D is deficient (below 20 ng/mL). Low Vitamin D can lead to bone weakness, fatigue, and weakened immunity. Your doctor may recommend supplements."),
            (25, "Your Vitamin D is insufficient (20-30 ng/mL). Consider Vitamin D supplementation and safe sun exposure."),
        ]
    },
    {
        "test": "TSH", "unit": "uIU/mL", "ref": "0.4-4.0",
        "scenarios": [
            (2.0, "Your TSH level is in the normal range, indicating your thyroid gland is functioning properly. The thyroid controls your metabolism and energy levels."),
            (0.2, "Your TSH is low, which may indicate hyperthyroidism (overactive thyroid). This can cause anxiety, rapid heartbeat, and weight loss. Further testing recommended."),
            (6.5, "Your TSH is elevated, suggesting hypothyroidism (underactive thyroid). This can cause fatigue, weight gain, and cold intolerance."),
        ]
    },
    {
        "test": "Total Cholesterol", "unit": "mg/dL", "ref": "<200",
        "scenarios": [
            (175, "Your total cholesterol is in the desirable range. This is good for heart health. Continue eating a balanced diet low in saturated fats."),
            (220, "Your cholesterol is borderline high (200-239 mg/dL). This increases your risk for heart disease. Consider dietary changes and regular exercise."),
            (260, "Your cholesterol is high (240+ mg/dL), which significantly increases heart disease risk. Your doctor will likely recommend lifestyle changes and possibly medication."),
        ]
    },
    {
        "test": "LDL Cholesterol", "unit": "mg/dL", "ref": "<100",
        "scenarios": [
            (85, "Your LDL (bad cholesterol) is optimal. This reduces your risk of heart disease and stroke."),
            (130, "Your LDL is borderline high (130-159 mg/dL). LDL can build up in arteries. Focus on heart-healthy diet and regular exercise."),
            (170, "Your LDL is high (160-189 mg/dL). This puts you at increased risk for cardiovascular disease. Lifestyle changes and medication may be needed."),
        ]
    },
    {
        "test": "HDL Cholesterol", "unit": "mg/dL", "ref": ">40 (men), >50 (women)",
        "scenarios": [
            (60, "Your HDL (good cholesterol) is at a protective level. HDL helps remove bad cholesterol from your arteries."),
            (35, "Your HDL is low, which reduces protection against heart disease. Exercise regularly and eat healthy fats to raise HDL."),
        ]
    },
    {
        "test": "Triglycerides", "unit": "mg/dL", "ref": "<150",
        "scenarios": [
            (120, "Your triglycerides are in the normal range, indicating good metabolic health."),
            (200, "Your triglycerides are borderline high (150-199 mg/dL). Reduce sugar and refined carbs, limit alcohol, and exercise more."),
            (350, "Your triglycerides are high (200-499 mg/dL). This increases risk for heart disease. Lifestyle changes are crucial."),
        ]
    },
    {
        "test": "Creatinine", "unit": "mg/dL", "ref": "0.7-1.3",
        "scenarios": [
            (1.0, "Your creatinine level is normal, indicating healthy kidney function. The kidneys are working well to filter waste."),
            (1.8, "Your creatinine is elevated, which may indicate reduced kidney function. Your doctor may order additional tests."),
        ]
    },
    {
        "test": "ALT (Liver Enzyme)", "unit": "U/L", "ref": "7-56",
        "scenarios": [
            (30, "Your ALT liver enzyme is in the normal range, indicating healthy liver function."),
            (85, "Your ALT is mildly elevated, suggesting some liver inflammation or stress. Your doctor may recommend lifestyle changes or further testing."),
            (150, "Your ALT is significantly elevated, indicating liver damage or inflammation. This requires further investigation."),
        ]
    },
    {
        "test": "Hemoglobin", "unit": "g/dL", "ref": "13.5-17.5 (men), 12.0-15.5 (women)",
        "scenarios": [
            (14.5, "Your hemoglobin is normal. Hemoglobin carries oxygen in your blood. Your level indicates good oxygen-carrying capacity."),
            (10.5, "Your hemoglobin is low, indicating anemia. This can cause fatigue, weakness, and pale skin. Iron supplements or dietary changes may help."),
        ]
    },
    {
        "test": "White Blood Cell Count", "unit": "x10^9/L", "ref": "4.0-11.0",
        "scenarios": [
            (7.5, "Your white blood cell count is normal. White blood cells fight infection and are part of your immune system."),
            (2.8, "Your WBC is low (leukopenia), which may increase infection risk. Inform your doctor if you develop fever or signs of infection."),
            (15.0, "Your WBC is elevated (leukocytosis). This usually indicates your body is fighting an infection or inflammation."),
        ]
    },
    {
        "test": "Platelet Count", "unit": "x10^9/L", "ref": "150-400",
        "scenarios": [
            (250, "Your platelet count is normal. Platelets help blood clot and prevent bleeding."),
            (95, "Your platelet count is low (thrombocytopenia). This can increase bleeding risk. Avoid activities that could cause injury."),
        ]
    },
]

# Generate synthetic examples
for template in synthetic_templates:
    for result, explanation in template["scenarios"]:
        formatted = format_for_training(
            test_name=template["test"],
            result=result,
            unit=template["unit"],
            ref_range=template["ref"],
            explanation=explanation
        )
        training_data.append(formatted)

print(f"✅ Added {len(training_data) - len(df)} synthetic examples")

# ============================================================================
# STEP 4: Shuffle and Save
# ============================================================================

print("\n💾 Saving dataset...")

random.shuffle(training_data)

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(training_data, f, ensure_ascii=False, indent=2)

print(f"✅ Saved {len(training_data)} examples to {OUTPUT_JSON}")




🧪 Creating synthetic examples...
✅ Added 78 synthetic examples

💾 Saving dataset...
✅ Saved 105 examples to /content/drive/MyDrive/lab_reports_training_dataset.json


In [19]:
# ============================================================================
# STEP 5: Show Statistics
# ============================================================================

print("\n" + "=" * 80)
print("Dataset Statistics")
print("=" * 80)
print(f"Original examples: {len(df)}")
print(f"Synthetic examples: {len(training_data) - len(df)}")
print(f"Total examples: {len(training_data)}")

print("\n📋 Sample Examples:")
print("=" * 80)
for i, example in enumerate(training_data[:3], 1):
    print(f"\nExample {i}:")
    print(f"Input: {example['instruction']}")
    print(f"Output: {example['output'][:150]}...")
    print("-" * 80)

print("\n✨ Dataset preparation complete!")
print(f"📁 File created: {OUTPUT_JSON}")
print("\n🚀 Next step: python finetune.py")


Dataset Statistics
Original examples: 27
Synthetic examples: 78
Total examples: 105

📋 Sample Examples:

Example 1:
Input: Explain this lab test result in simple terms: Test: White Blood Cell Count | Result: 15.0 x10^9/L (Reference: 4.0-11.0)
Output: Your WBC is elevated (leukocytosis). This usually indicates your body is fighting an infection or inflammation....
--------------------------------------------------------------------------------

Example 2:
Input: Explain this lab test result in simple terms: Test: Platelet | Result: 267 10^3/uL (Reference: 150-450)
Output: Platelet count is normal Routine check...
--------------------------------------------------------------------------------

Example 3:
Input: Explain this lab test result in simple terms: Test: Platelet Count | Result: 520 x10^9/L (Reference: 150-400)
Output: Your platelet count is elevated (thrombocytosis). This can increase blood clot risk. Causes include inflammation, iron deficiency, or bone marrow diso...
-------

In [20]:
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
# ============================================================================
# Fine-tune the model
# ============================================================================


import json
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# ============================================================================
# CONFIGURATION
# ============================================================================

# Paths
DATASET_PATH = "/content/drive/MyDrive/lab_reports_training_dataset.json"
OUTPUT_MODEL_DIR = "/content/drive/MyDrive/lab_report_explainer_model"

# Model settings
MODEL_NAME = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True

# Training hyperparameters
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4
MAX_STEPS = 100
WARMUP_STEPS = 10
SAVE_STEPS = 25

# LoRA settings
LORA_RANK = 16
LORA_ALPHA = 16

# ============================================================================
# Load Dataset
# ============================================================================

print("=" * 80)
print("STEP 1: Loading Prepared Dataset")
print("=" * 80)

with open(DATASET_PATH, "r", encoding="utf-8") as f:
    training_data = json.load(f)

print(f"✅ Loaded {len(training_data)} training examples")
print(f"\nSample example:")
print(f"Instruction: {training_data[0]['instruction']}")
print(f"Output: {training_data[0]['output'][:150]}...")

# ============================================================================
# STEP 2: Load Base Model
# ============================================================================

print("\n" + "=" * 80)
print("STEP 2: Loading Base Model")
print("=" * 80)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=LOAD_IN_4BIT,
)

print(f"✅ Loaded model: {MODEL_NAME}")



STEP 1: Loading Prepared Dataset
✅ Loaded 105 training examples

Sample example:
Instruction: Explain this lab test result in simple terms: Test: White Blood Cell Count | Result: 15.0 x10^9/L (Reference: 4.0-11.0)
Output: Your WBC is elevated (leukocytosis). This usually indicates your body is fighting an infection or inflammation....

STEP 2: Loading Base Model
==((====))==  Unsloth 2025.11.2: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Loaded model: unsloth/Phi-3-mini-4k-instruct-bnb-4bit


In [ ]:
# ============================================================================
# STEP 3: Add LoRA Adapters
# ============================================================================

print("\n" + "=" * 80)
print("STEP 3: Adding LoRA Adapters for Efficient Fine-tuning")
print("=" * 80)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print(f"✅ Added LoRA adapters (rank={LORA_RANK}, alpha={LORA_ALPHA})")

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"📊 Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")

# ============================================================================
# Format Dataset for Training
# ============================================================================

print("\n" + "=" * 80)
print("STEP 4: Formatting Dataset")
print("=" * 80)

def formatting_func(examples):
    """Format examples into training prompts"""
    texts = []
    for instruction, output in zip(examples["instruction"], examples["output"]):
        text = f"""### Instruction:
{instruction}

### Response:
{output}"""
        texts.append(text)
    return {"text": texts}

# Create HuggingFace compatible dataset
dataset = Dataset.from_list(training_data)
dataset = dataset.map(formatting_func, batched=True)

print(f"✅ Formatted {len(dataset)} examples for training")
print(f"\nSample formatted text:")
print(dataset[0]["text"][:300] + "...")

# ============================================================================
# Configure Trainer
# ============================================================================

print("\n" + "=" * 80)
print("STEP 5: Configuring Trainer")
print("=" * 80)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=WARMUP_STEPS,
        max_steps=MAX_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=SAVE_STEPS,
        report_to="none",  
    ),
)

print("✅ Trainer configured")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"   Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Max steps: {MAX_STEPS}")


STEP 3: Adding LoRA Adapters for Efficient Fine-tuning


Unsloth 2025.11.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ Added LoRA adapters (rank=16, alpha=16)
📊 Trainable parameters: 29,884,416 (1.47%)

STEP 4: Formatting Dataset


Map:   0%|          | 0/105 [00:00<?, ? examples/s]

✅ Formatted 105 examples for training

Sample formatted text:
### Instruction:
Explain this lab test result in simple terms: Test: White Blood Cell Count | Result: 15.0 x10^9/L (Reference: 4.0-11.0)

### Response:
Your WBC is elevated (leukocytosis). This usually indicates your body is fighting an infection or inflammation....

STEP 5: Configuring Trainer


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/105 [00:00<?, ? examples/s]

✅ Trainer configured
   Batch size: 2
   Gradient accumulation: 4
   Effective batch size: 8
   Learning rate: 0.0002
   Max steps: 100


In [ ]:
# ============================================================================
# Now Fine-tune Model with configured trainer
# ============================================================================

print("\n" + "=" * 80)
print("STEP 6: Starting Fine-tuning")
print("=" * 80)
print("This may take a while depending on your hardware and MAX_STEPS...")
print("")

trainer_stats = trainer.train()

print("\n✅ Fine-tuning complete!")
print(f"\n📊 Training Statistics:")
print(f"   Total training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"   Final loss: {trainer_stats.metrics['train_loss']:.4f}")



The model is already on multiple devices. Skipping the move to device specified in `args`.



STEP 6: Starting Fine-tuning
This may take a while depending on your hardware and MAX_STEPS...



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 105 | Num Epochs = 8 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Step,Training Loss
10,1.561400
20,0.733500
30,0.487800
40,0.367800
50,0.257100
60,0.243600
70,0.179700
80,0.140600
90,0.128700
100,0.106100



✅ Fine-tuning complete!

📊 Training Statistics:
   Total training time: 213.69 seconds
   Final loss: 0.4206


In [6]:
# ============================================================================
# STEP 7: Save Fine-tuned Model
# ============================================================================

print("\n" + "=" * 80)
print("STEP 7: Saving Fine-tuned Model")
print("=" * 80)

model.save_pretrained(OUTPUT_MODEL_DIR)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)

print(f"✅ Model saved to: {OUTPUT_MODEL_DIR}/")


STEP 7: Saving Fine-tuned Model
✅ Model saved to: /content/drive/MyDrive/lab_report_explainer_model/


In [7]:
# ============================================================================
# STEP 8: Test the Model
# ============================================================================

print("\n" + "=" * 80)
print("STEP 8: Testing Fine-tuned Model")
print("=" * 80)

FastLanguageModel.for_inference(model)

test_cases = [
    "Test: Glucose (Fasting) | Result: 145 mg/dL (Reference: 70-100)",
    "Test: Vitamin D | Result: 18 ng/mL (Reference: 30-100)",
    "Test: TSH | Result: 5.5 uIU/mL (Reference: 0.4-4.0)",
    "Test: Total Cholesterol | Result: 245 mg/dL (Reference: <200)",
    "Test: Hemoglobin | Result: 10.2 g/dL (Reference: 13.5-17.5)",
]

print("\n🧪 Testing with sample lab results:\n")

for i, test_input in enumerate(test_cases, 1):
    prompt = f"""### Instruction:
Explain this lab test result in simple terms: {test_input}

### Response:
"""

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract response
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()

    print(f"\n{'='*80}")
    print(f"Test Case {i}")
    print(f"{'='*80}")
    print(f"📋 Input: {test_input}")
    print(f"\n💬 Explanation:")
    print(f"   {response}")



STEP 8: Testing Fine-tuned Model

🧪 Testing with sample lab results:


Test Case 1
📋 Input: Test: Glucose (Fasting) | Result: 145 mg/dL (Reference: 70-100)

💬 Explanation:
   Your LDL (bad cholesterol) is borderline high. Lifestyle changes like diet, exercise, and weight management can help.

Test Case 2
📋 Input: Test: Vitamin D | Result: 18 ng/mL (Reference: 30-100)

💬 Explanation:
   Your Vitamin D is insufficient (20-30 ng/mL). Low Vitamin D can lead to bone weakness and fatigue. Consider Vitamin D supplementation and safe sun exposure. Follow up with your doctor. More info: https://www.nlm.nih.gov/medline/Drugreference/d630 Report this test: URATE | Result: 7.2 mg/dL (Reference: 2.6-6.6) Medical Professional Insight: Your uric acid is high (6.0-8.0 mg/dL). This may cause gout. Your doctor may recommend lifestyle changes or medication. More info: https://www.nlm.nih.gov/medline/Drugreference/d631 Report this test: BUN | Result: 18 mg/dL (Reference: 7-20) Medical Professional Insigh

In [1]:
# Save in different format
# I will use gguf to serve it with ollama

# # 1. Save merged model (LoRA weights merged into base model)
# model.save_pretrained_merged(
#     f"{OUTPUT_MODEL_DIR}_merged",
#     tokenizer,
#     save_method="merged_16bit",
# )
# print(f"✅ Merged model saved to: {OUTPUT_MODEL_DIR}_merged/")

# 2. Save GGUF format (for llama.cpp )
from huggingface_hub import login

try:
    #pushing the model to hugging_face hub
    login()
    model.push_to_hub_gguf(
        "dat_hassan/Lab-Assistant-gguf",
        tokenizer,
        quantization_method="q4_k_m",
    )

    #if you have enough disk space use this
    # model.save_pretrained_gguf(
    #     f"{OUTPUT_MODEL_DIR}_gguf",
    #     tokenizer,
    #     quantization_method="q4_k_m",
    # )
    print("✅ model uploaded to hugging_face hub")
except Exception as e:
    print(f"⚠️  GGUF export skipped: {e}")

⚠️  GGUF export skipped: name 'model' is not defined
